----

# 🤖 Transfer Learning - Transferência de Aprendizado 🤖

----

A Transferência de aprendizado é baseada na aplicação do conhecimento aprendido no processo de treinamento de uma Rede Neural Convolucional (CNN), de um determinado problema para outro diferente, porém, semelhante. Aqui utilizamos quatro redes neurais pré-treinadas, comuns em problemas de classificação de imagens: ResNet50, VGG16, InceptionV3 e MobileNetV2.

As redes são carregadas com pesos ImageNet, sendo a útima camada de classificação removida, e uma camada de GlobalAveragepooling adicionada para formar o extrator de características das imagens. As características extraídas são salvas em um arquivo .npz, para serem utilizadas como vetores de entrada nos algoritmos de machine learning.

Mais detalhes sobre cada uma das redes, podem sera acessados através do link: [Keras Applications](https://keras.io/api/applications/)

Conectando ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **1- Importando Bibliotecas**

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.models import load_model, save_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet50 import preprocess_input
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Model

# **2- Extraindo características das imagens inteiras**

## **Carregando dados**

In [ ]:
# ============================================================================================================================
# Carregar arquivos com caminhos e nomes das espécies (Mapeia os nomes pelos códigos)
# ============================================================================================================================
df = pd.read_csv('./Amazon_Bark_Split/filepath.csv')
labels = pd.read_csv('./Class.csv', sep = ';')
df['Specie'] = df['Class'].map(labels.set_index('Code')['Specie'])

In [ ]:
#------------------------------------------------------------------------------#
# Separandos dados, labels e amostras
#------------------------------------------------------------------------------#
train = df.loc[df['Set'] == 'train']                                                        # Filtra linhas "train"
test = df.loc[df['Set'] == 'test']                                                          # Filtra linhas "test"

# Labels
y_train = df.loc[df['Set'] == 'train', 'Specie']
y_test = df.loc[df['Set'] == 'test', 'Specie']

# Samples
sample_train = df.loc[df['Set'] == 'train', 'Sample']
sample_test = df.loc[df['Set'] == 'test', 'Sample']

In [ ]:
# ============================================================================================================================
# Carregar lotes de imagens - (ImageDataGenerator + flow_from_dataframe)
# ============================================================================================================================
# Pré processamento
train_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()

train_generator = train_gen.flow_from_dataframe(dataframe = train,
                                                directory = './Amazon_Bark_Split/train',
                                                x_col = 'filename',
                                                y_col = 'Specie',
                                                class_mode = "sparse",
                                                target_size = (256, 256),
                                                batch_size = 32,
                                                seed = 42,
                                                shuffle = False
                                                )

test_generator = test_gen.flow_from_dataframe(dataframe = test,
                                              directory = './Amazon_Bark_Split/test',
                                              x_col = 'filename',
                                              y_col = 'Specie',
                                              class_mode = "sparse",
                                              target_size = (256, 256),
                                              batch_size = 32,
                                              seed = 42,
                                              shuffle = False
                                              )

Found 2237 validated image filenames belonging to 16 classes.
Found 566 validated image filenames belonging to 16 classes.


## **ResNet50**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo ResNet50 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento ResNet
#-----------------------------------------------------------------------------#
preprocess_resnet = tf.keras.applications.resnet50.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features ResNet
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_resnet(inputs)
x = base_model(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, output)

# Visualizar o resumo do modelo
feature_extractor.summary()

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# Train
X_train_features = feature_extractor.predict(train_generator)

# Test
X_test_features = feature_extractor.predict(test_generator)

18/18 [==============================] - 390s 23s/step


In [ ]:
#------------------------------------------------------------------------------#
# Salvar features
#------------------------------------------------------------------------------#

np.savez(f'./Output/features_train.npz', X_train = X_train_features, y_train = y_train, sample_train = sample_train)
np.savez(f'./Output/features_test.npz', X_test = X_test_features, y_test = y_test, sample_test = sample_test)

## **VGG16**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo VGG16 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento VGG16
#-----------------------------------------------------------------------------#
preprocess_vgg16 = tf.keras.applications.vgg16.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features VGG16
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_vgg16(inputs)
x = base_model(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, outputs=output)

# Visualizar o resumo do modelo
feature_extractor.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem_1  (None, 224, 224, 3)       0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_1 (TFOpLamb  (None, 224, 224, 3)       0         
 da)                                                             
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                           

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# Train
X_train_features = feature_extractor.predict(train_generator)

# Test
X_test_features = feature_extractor.predict(test_generator)

18/18 [==============================] - 443s 26s/step


In [ ]:
#------------------------------------------------------------------------------#
# Salvar features
#------------------------------------------------------------------------------#

np.savez(f'./Output/CNN/Features/features_vgg16_train.npz', X_train = X_train_features, y_train = y_train, sample_train = sample_train)
np.savez(f'./Output/CNN/Features/features_vgg_16test.npz', X_test = X_test_features, y_test = y_test, sample_test = sample_test)

## **Inception**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo InceptionV3 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento InceptionV3
#-----------------------------------------------------------------------------#
preprocess_inception = tf.keras.applications.inception_v3.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features InceptionV3
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_inception(inputs)
x = base_model(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, outputs=output)

# Visualizar o resumo do modelo
feature_extractor.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLam  (None, 224, 224, 3)       0         
 bda)                                                            
                                                                 
 tf.math.subtract_1 (TFOpLa  (None, 224, 224, 3)       0         
 mbda)                                                           
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                           

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# Train
X_train_features = feature_extractor.predict(train_generator)

# Test
X_test_features = feature_extractor.predict(test_generator)

18/18 [==============================] - 146s 8s/step


In [ ]:
#------------------------------------------------------------------------------#
# Salvar features
#------------------------------------------------------------------------------#

np.savez(f'./Output/CNN/Features/features_inception_train.npz', X_train = X_train_features, y_train = y_train, sample_train = sample_train)
np.savez(f'./Output/CNN/Features/features_inception_16test.npz', X_test = X_test_features, y_test = y_test, sample_test = sample_test)

## **MobileNet_V2**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo MobileNetV2 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento MobileNetV2
#-----------------------------------------------------------------------------#
preprocess_mobile = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features MobileNetV2
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_mobile(inputs)
x = base_model(x)
#output = Flatten()(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, outputs=output)

# Visualizar o resumo do modelo
feature_extractor.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_3 (TFOpLam  (None, 224, 224, 3)       0         
 bda)                                                            
                                                                 
 tf.math.subtract_3 (TFOpLa  (None, 224, 224, 3)       0         
 mbda)                                                           
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                 

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# Train
X_train_features = feature_extractor.predict(train_generator)

# Test
X_test_features = feature_extractor.predict(test_generator)

18/18 [==============================] - 152s 9s/step


In [ ]:
#------------------------------------------------------------------------------#
# Salvar features
#------------------------------------------------------------------------------#

np.savez(f'./Output/CNN/Features/features_mobnet_train.npz', X_train = X_train_features, y_train = y_train, sample_train = sample_train)
np.savez(f'./Output/CNN/Features/features_mobnet_test.npz', X_test = X_test_features, y_test = y_test, sample_test = sample_test)

# **4- Extraindo caracteristicas dos patches**

## **Carregando dados**

A base de dados dos patches possui 111.850 patches de treinamento, e 28.300 patches de teste. Para otimizar o processamento foi utilizada uma estratégia de distribuição de tpu.

In [ ]:
#===============================================================================
# Extratégia de distribuição de TPU para patches
#===============================================================================
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print(f'Running on a TPU w/{tpu.num_accelerators()["TPU"]} cores')
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.15.0
Running on a TPU w/8 cores


In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o arquivo .npz
#-----------------------------------------------------------------------------#
with np.load('./Output_patches/patches_and_labels_train.npz', mmap_mode= 'r', allow_pickle=True) as f:
    X_train = f['patches']
    y_train = f['labels']

with np.load('./Output_patches/patches_and_labels_test.npz', mmap_mode= 'r', allow_pickle=True) as f:
    X_test = f['patches']
    y_test = f['labels']

In [ ]:
#-----------------------------------------------------------------------------#
# Obtendo labels - Train
#-----------------------------------------------------------------------------#
lab = []
for label in range(len(y_train)):
  split = y_train[label].split('_')
  path = split[0]
  specie = int(split[2])
  sample = int(split[4])
  code = split[6].split('.')[0]
  dev = split[5]
  lab.append([path, specie, sample, code, dev])

labels_train = pd.DataFrame(lab, columns= ['Patch','Specie', 'Sample', 'Id', 'Dev'])
df = pd.read_csv('./Class.csv', sep = ';')
labels_train['Specie'] = labels_train['Specie'].map(df.set_index('Code')['Specie'])
labels_train

In [ ]:
#-----------------------------------------------------------------------------#
# Obtendo labels - Test
#-----------------------------------------------------------------------------#
lab = []
for label in range(len(y_test)):
  split = y_test[label].split('_')
  path = split[0]
  specie = int(split[2])
  sample = int(split[4])
  code = split[6].split('.')[0]
  dev = split[5]
  lab.append([path, specie, sample, code, dev])

labels_test = pd.DataFrame(lab, columns= ['Patch','Specie', 'Sample', 'Id', 'Dev'])
df = pd.read_csv('./Class.csv', sep = ';')
labels_test['Specie'] = labels_test['Specie'].map(df.set_index('Code')['Specie'])
labels_test

## **ResNet50**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo ResNet50 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features ResNet
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_resnet(inputs)
x = base_model(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, output)

# Visualizar o resumo do modelo
feature_extractor.summary()

In [ ]:
# train
with tpu_strategy.scope():
  # Extrair características das imagens
  X_train_resize = tf.image.resize(X_train, (224, 224))
  X_train_preprocessed = preprocess_resnet(X_train_resize)
  X_train_features = feature_extractor.predict(X_train_preprocessed)

# test
with tpu_strategy.scope():
  # Extrair características das imagens
  X_test_resize = tf.image.resize(X_test, (224, 224))
  X_test_preprocessed = preprocess_resnet(X_test_resize)
  X_test_features = feature_extractor.predict(X_test_preprocessed)

885/885 [==============================] - 157s 177ms/step


In [ ]:
np.savez(f'./Output_patches/Features/patches_resnet_train.npz', X_train = X_train_features, y_train = labels_train)
np.savez(f'./Output_patches/Features/patches_resnet_test.npz', X_test = X_test_features, y_test = labels_test)

## **VGG16**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo VGG16 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento VGG16
#-----------------------------------------------------------------------------#
preprocess_vgg16 = tf.keras.applications.vgg16.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features VGG16
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_vgg16(inputs)
x = base_model(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, outputs=output)

# Visualizar o resumo do modelo
feature_extractor.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         
 )                                                               
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                             

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# train
with tpu_strategy.scope():
  # Extrair características das imagens
  X_train_resize = tf.image.resize(X_train, (224, 224))
  X_train_features = feature_extractor.predict(X_train_resize)

#test
with tpu_strategy.scope():
  # Extrair características das imagens
  X_test_resize = tf.image.resize(X_test, (224, 224))
  X_test_features = feature_extractor.predict(X_test_resize)

885/885 [==============================] - 64s 72ms/step


In [ ]:
np.savez(f'./Output_patches/CNN/Features/patches_vgg16_train.npz', X_train = X_train_features, y_train = labels_train)
np.savez(f'./Output_patches/CNN/Features/patches_vgg16_test.npz', X_test = X_test_features, y_test = labels_test)

## **Inception**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo InceptionV3 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento InceptionV3
#-----------------------------------------------------------------------------#
preprocess_inception = tf.keras.applications.inception_v3.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features InceptionV3
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_inception(inputs)
x = base_model(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, outputs=output)

# Visualizar o resumo do modelo
feature_extractor.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 224, 224, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 224, 224, 3)       0         
 da)                                                             
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                           

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# train
with tpu_strategy.scope():
  # Extrair características das imagens
  X_train_resize = tf.image.resize(X_train, (224, 224))
  X_train_features = feature_extractor.predict(X_train_resize)

#test
with tpu_strategy.scope():
  # Extrair características das imagens
  X_test_resize = tf.image.resize(X_test, (224, 224))
  X_test_features = feature_extractor.predict(X_test_resize)

885/885 [==============================] - 207s 233ms/step


In [ ]:
#------------------------------------------------------------------------------#
# Salvar features
#------------------------------------------------------------------------------#

np.savez(f'./Output_patches/CNN/Features/patches_inception_train.npz', X_train = X_train_features, y_train = labels_train)
np.savez(f'./Output_patches/CNN/Features/patches_inception_test.npz', X_test = X_test_features, y_test = labels_test)

## **MobileNet_V2**

In [ ]:
#-----------------------------------------------------------------------------#
# Carregar o modelo MobileNetV2 pré-treinado com pesos ImageNet
#-----------------------------------------------------------------------------#
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
#-----------------------------------------------------------------------------#
# Pré processamento MobileNetV2
#-----------------------------------------------------------------------------#
preprocess_mobile = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
#-----------------------------------------------------------------------------#
# Construindo extrator de features MobileNetV2
#-----------------------------------------------------------------------------#
inputs = keras.Input(shape=(224, 224, 3))
x = preprocess_mobile(inputs)
x = base_model(x)
#output = Flatten()(x)
output = GlobalAveragePooling2D()(x)

# Criar um novo modelo para extração de características
feature_extractor = Model(inputs, outputs=output)

# Visualizar o resumo do modelo
feature_extractor.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLam  (None, 224, 224, 3)       0         
 bda)                                                            
                                                                 
 tf.math.subtract_1 (TFOpLa  (None, 224, 224, 3)       0         
 mbda)                                                           
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                 

In [ ]:
#------------------------------------------------------------------------------#
# Extrair características das imagens
#------------------------------------------------------------------------------#
# train
with tpu_strategy.scope():
  # Extrair características das imagens
  X_train_resize = tf.image.resize(X_train, (224, 224))
  X_train_features = feature_extractor.predict(X_train_resize)

#test
with tpu_strategy.scope():
  # Extrair características das imagens
  X_test_resize = tf.image.resize(X_test, (224, 224))
  X_test_features = feature_extractor.predict(X_test_resize)

885/885 [==============================] - 113s 127ms/step


In [ ]:
#------------------------------------------------------------------------------#
# Salvar features
#------------------------------------------------------------------------------#

np.savez(f'./Output_patches/CNN/Features/patches_mobnet_train.npz', X_train = X_train_features, y_train = labels_train)
np.savez(f'./Output_patches/CNN/Features/patches_mobnet_test.npz', X_test = X_test_features, y_test = labels_test)